# Movie genre multiclass classification problem using softmax and bookeh

Firsts things firsts, this notebook will use the data distributed by Kaggle: https://www.kaggle.com/shainy/twitter-reviews-for-emotion-analysis
<br />
Resuming, it's about movie genre
<br /><br />
In this notebook, we will try to predict the movie genre using the **softmax function** and after, plotting using a bokeh graph

----------------------------------------------------------------------------------------------------------------------------------------------------------------

## Beginning of the work

To begin, we will mount the connection with drive so we can acess the csv

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Besides, we will import and use **sys**, so we can import the functions that we are going to use of a **.py** file

In [2]:
import sys
sys.path.append('/content/drive/My Drive/Tarefinha de Data Science/Arquivos - Carlos')

## Imports

Before starting to use the data, we will import all the things we need


In [1]:
# dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
# pd.set_option('display.max_colwidth', -1)

# my functions
# import UtilsCarlos
# from UtilsCarlos import  criaDicio, criaVetor, fit_and_score, plot_conf_mat, convert

# nltk
import nltk
from nltk.tokenize import TweetTokenizer
import re
from nltk import FreqDist
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords 
tknzr = TweetTokenizer()
stemmer = PorterStemmer()
nltk.download('punkt')

# modelos
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import tree

# evaluations
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import plot_roc_curve
from sklearn.model_selection import cross_val_score

# others
import json
import string

# PyTorch
#import torch
#from torch import nn

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\carlo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words("english")
token_espaço = nltk.tokenize.WhitespaceTokenizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


---------------------------------------------------------------------------------------------------------

In [5]:
# função de limpar texto
def cleanText(words, stem=False):
  """
    Esta função recebe um text e retorna o mesmo, já tratado com stopwords & punctuation
  """
  newWords = list()
  pontuacao = string,
  for word in words:
    word = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', word)
    words = re.sub("(@[A-Za-z0-9_]+)","", word)
    if len(word) > 0 and words not in string.punctuation and word.lower() not in stopwords and word.lower != "<br />":
      if stem:
        word = stemmer.stem(word.lower())
        newWords.append(word)
      else:
        newWords.append(word.lower())

  return newWords

In [6]:
# confusion matrix daora
def plot_conf_mat(y_test, y_preds, norm="false"):
    fig, ax = plt.subplots(figsize=(3, 3))
    ax = sns.heatmap(confusion_matrix(y_test, y_preds, normalize=norm),
                    annot=True,
                    cbar=False)
    plt.xlabel("True label")
    plt.ylabel("Predicted label")

In [7]:
def plot_loss_and_accuracy(losses, accs):

  fig, ax_tuple = plt.subplots(1, 2, figsize=(16,6))
  fig.suptitle('Loss and accuracy')

  for i, (y_label, y_values) in enumerate(zip(['BCE loss','Accuracy'],[losses, accs])):
    ax_tuple[i].plot(range(len(y_values)),  y_values, label='train')
    ax_tuple[i].set_xlabel('epochs')
    ax_tuple[i].set_ylabel(y_label)
    ax_tuple[i].legend()

In [4]:
def label2Embedding(sentence):
  for word in sentence: 
    if word in embedding.vocab:
      embed = embedding.get_vector(word)
      return embed
    else:
      print("This word is not in the vocabuylary: ", word, "\n")

In [2]:
filename_txt = '../data/glove.6B.50d.txt'

with open(filename_txt, 'r+', encoding="utf8") as f:
  content = f.read()
  for i, l in enumerate(f):
    pass
    line = f'{i+1} 100'
    f.seek(0, 0)
    f.write(line.rstrip('\r\n') + '\n' + content)

with open(filename_txt) as f:
  for linha in range(10):
    print(next(f))

the 0.418 0.24968 -0.41242 0.1217 0.34527 -0.044457 -0.49688 -0.17862 -0.00066023 -0.6566 0.27843 -0.14767 -0.55677 0.14658 -0.0095095 0.011658 0.10204 -0.12792 -0.8443 -0.12181 -0.016801 -0.33279 -0.1552 -0.23131 -0.19181 -1.8823 -0.76746 0.099051 -0.42125 -0.19526 4.0071 -0.18594 -0.52287 -0.31681 0.00059213 0.0074449 0.17778 -0.15897 0.012041 -0.054223 -0.29871 -0.15749 -0.34758 -0.045637 -0.44251 0.18785 0.0027849 -0.18411 -0.11514 -0.78581

, 0.013441 0.23682 -0.16899 0.40951 0.63812 0.47709 -0.42852 -0.55641 -0.364 -0.23938 0.13001 -0.063734 -0.39575 -0.48162 0.23291 0.090201 -0.13324 0.078639 -0.41634 -0.15428 0.10068 0.48891 0.31226 -0.1252 -0.037512 -1.5179 0.12612 -0.02442 -0.042961 -0.28351 3.5416 -0.11956 -0.014533 -0.1499 0.21864 -0.33412 -0.13872 0.31806 0.70358 0.44858 -0.080262 0.63003 0.32111 -0.46765 0.22786 0.36034 -0.37818 -0.56657 0.044691 0.30392

. 0.15164 0.30177 -0.16763 0.17684 0.31719 0.33973 -0.43478 -0.31086 -0.44999 -0.29486 0.16608 0.11963 -0.41328 -0.423

In [3]:
# gensin
from gensim.models import KeyedVectors

modelo = KeyedVectors.load_word2vec_format(filename_txt)

ModuleNotFoundError: No module named 'gensim'

In [11]:
!unzip -o '/content/drive/My Drive/Tarefinha de Data Science/Arquivos - Carlos/glove.6B.zip'

Archive:  /content/drive/My Drive/Tarefinha de Data Science/Arquivos - Carlos/glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [13]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

file_name = 'glove.6B.100d.txt'
glove_file = datapath(file_name)
# tmp_file = get_tmpfile("test_word2vec.txt")

# _ = glove2word2vec(glove_file)

model = KeyedVectors.load_word2vec_format(glove_file)

FileNotFoundError: ignored

----------------------------------------------------------------------------------------------------------------------------------

In [ ]:
df = pd.read_csv("/content/drive/My Drive/Tarefinha de Data Science/Arquivos - Carlos/data.csv", encoding = "ISO-8859-1")
df.tail()

Creating a vocabulary

In [ ]:
dataset = df.copy()

In [ ]:
dataset.drop(columns=["Sl no", "Search key"]);

In [ ]:
dataset.head()

In [ ]:
dataset['Feeling'] = pd.Categorical(dataset['Feeling'])

In [ ]:
dataset['emotion_code'] = dataset['Feeling'].cat.codes
dataset.head()

## Cleaning texts

In [ ]:
dataset["CleanText"] = [tknzr.tokenize(word) for word in dataset["Tweets"]]

In [ ]:
dataset["CleanText"] = [cleanText(word) for word in dataset["CleanText"]]

In [ ]:
dataset.head()

In [ ]:
#Criando EL VOCABULÁRIO (Com ajuda do código do sor em: Introdução ao PyTorch: da Regressão Linear à NLP com word-embeddings)
vocab_set = set() # será usado para gerar o vocabulário principal
max_len_doc = 0   # vamos medir o maior comprimento das mensagens envidas, em número de tokens
sum_len_doc = 0   # vamos medir o valor médio de palavras (tokens) por mensagem
min_word_len = 3  # comprimento mínimo de um token (em número de caracteres) para entrar no vocabulário 
tokens_list = []  # salvar a lista de tokens

for doc in dataset['CleanText']: # para cada documento do dataset
  #Pegando a palavra apenas se ela é maior que o comprimento mínimo
  for word in doc: 
    if len(word)>=min_word_len:
      tokens_list.append(word)

  # uso da função set: cria um conjunto dos elementos únicos da lista
  tokens_set = set(tokens_list)
  vocab_set = set.union(vocab_set, tokens_set) # adiciona elementos únicos que ainda não pertencem ao conjunto do vocabulário

  sum_len_doc += len(word)
  if len(word) > max_len_doc:
    max_len_doc=len(word)

print(f'Tamanho total do vocabulário: V={len(vocab_set)}')
print(f'Número de palavras do texto mais longo: {max_len_doc}')
print(f'Média de palavras por texto: {sum_len_doc/len(df):3.4f}')

--------------------------------------------------------------------------------------

## Creating vocabulary based on trained word embeddding

In [ ]:
word2idx = dict({})        # inicializa o dicionário
word2idx['<OOV>'] = 0      # índice da tag "out of vocabulary" é 0
word2idx['<PAD>'] = 1      # índice da tag "padding token" é 1

for i, v in enumerate(sorted(vocab_set),start=2): # enumera o vocabulário em ordem alfabética, a partir do índice 2
  word2idx[v] = label2Embedding(v)

# testando a conversão "word to index" com o dicionário:
print(f'index for "<PAD>": {word2idx["<PAD>"]}')
print(f'index for "action": {word2idx["action"]}')

---------------------------------------------------------------------------------------

In [ ]:
#word2idx = dict({})        # inicializa o dicionário
#word2idx['<OOV>'] = 0      # índice da tag "out of vocabulary" é 0
#word2idx['<PAD>'] = 1      # índice da tag "padding token" é 1

#for i, v in enumerate(sorted(vocab_set),start=2): # enumera o vocabulário em ordem alfabética, a partir do índice 2
#  word2idx[v] = i

# testando a conversão "word to index" com o dicionário:
#print(f'index for "<PAD>": {word2idx["<PAD>"]}')
#print(f'index for "action": {word2idx["action"]}')

In [ ]:
idx2word = list(word2idx.keys()) # apenas transforma as chaves (palavras ordenadas) do dicionário word2idx em uma lista

# testando a conversão "index to word":
print(f'word for index 0:    {idx2word[0]}')
print(f'word for index 100": {idx2word[100]}')

Coding the texts

In [ ]:
max_len = 25         # comprimento máximo da mensagem (em número de palavras)
encoded_docs = []    # inicializa a lista de documentos codificados

for doc in dataset['CleanText']: # para cada texto
  encoded_d = [word2idx.get(t,word2idx['<OOV>']) for t in doc]    # codifica o documento usando o dicionário word2idx
  encoded_d += [word2idx['<PAD>']]*max(0, max_len-len(encoded_d))    # adiciona o padding, se necessário
  
  encoded_docs.append(encoded_d[:max_len])                           # trunca o documento e salva na lista de documentos codificados

len(encoded_docs)  

In [ ]:
dataset['CleanText'] = encoded_docs
dataset.head()

Splitting into Train and Test

In [ ]:
X = np.vstack(dataset['CleanText'].apply(lambda x: np.array(x)))
Y = np.array(dataset['emotion_code']).reshape(-1,1)
X.shape, X[0].shape, Y.shape, Y[0].shape

In [ ]:
train_size = 0.8    # percentual de exemplos para o treino

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,                       # dataset para ser dividido, entrada X e saída Y
                                                    train_size=train_size,     # percentual resevado para o treinamento
                                                    stratify=Y,                # estratificação para manter a distribuição dos rótulos igual entre treino e teste
                                                    shuffle=True)              # embaralhar os exemplos aleatoriamente

1. **Seguir o tutorial pra ter acesso aos embedding de cada palavra**

2. Converter as palavras para o próprio embedding durante o treinamento (1 frase por vez) -- criar funçao pra tal
  - usar  embedding.get_vector(palavra)

3. pyTortch dataLoader - batchSize --> https://pytorch.org/docs/stable/data.html


In [ ]:
#words = ["draw", "to", "the", "ground"]
#teste = [label2Embedding(word) for word in words]
#teste

Creating model

In [ ]:
class Torch_Mean_Layer(nn.Module):
  '''Camada personalizada: calcula a média do tensor dentrada sobre a dimensão 1 (colunas).
     Retorna um vetor linha, onde cada elemento é a média dos elementos da coluna correspondente do tensor de entrada.
  '''
  def forward(self, x, dim=1):
    print("-----------------------",x)
    x = torch.mean(x, dim=dim, keepdims=True)
    return x

class mood_classifier(nn.Module):
  '''Modelo classificador de emoções
  '''

  # ----------------------------------------------#
  # Método construtor
  def __init__(self, vocab_size, dim_embed, n_units): 
    super().__init__()  

    embedding_seq = [] # 
    ann_seq       = [] # 
    soft_seq      = []

    #---------------------------------------------------------------#
    # Embedding step: sequência de operações para converter X --> h
    embedding_seq.append(Torch_Mean_Layer())
    #---------------------------------------------------------------#

    #--------------------------------------------------------------------------#
    # ANN: Rede Neural Artifical Tradicional, com regressão logística na saída
    ann_seq.append(nn.Linear(dim_embed, n_units))
    ann_seq.append(nn.ReLU(inplace=True))
    ann_seq.append(nn.Linear(n_units, 6))
    
    #--------------------------------------------------------------------------#
    # Softmax :)
    soft_seq.append(nn.LogSoftmax(dim=1))

    #--------------------------------------------------------------------------#

    #--------------------------------------------------------------------------#
    # "merge" de todas as camamadas em uma layer sequencial 
    # (uma sequência para cada etapa)
    self.embedding = nn.Sequential(*embedding_seq)     # etapa de embedding 
    self.ann       = nn.Sequential(*ann_seq)           # etapa ANN
    self.soft      = nn.Sequential(*soft_seq)
    #--------------------------------------------------------------------------#


  def forward(self, x): 
    '''Processamento realizado ao chamar y=modelo(x)
    '''
    x = self.embedding(x)  # aplica a etapa de embedding
    x = self.ann(x)        # passa o embedding médio pelas camadas da ANN
    x = x.view(-1,6)
    x = self.soft(x)
    return x  #Adcionar o softmax

Training model

In [ ]:
import tensorflow as tf
def train_loop(model, data, max_epochs = 1000, print_iters = 5):
  X_train, Y_train = data
  losses = []
  accs = []
  for i in range(max_epochs): # para cada época

      #-----------------------------------#
      # INÍCIO DO WORKFLOW DO TREINAMENTO #
      # 
      # Add mistura

      Y_pred = model.forward(X_train)         # apresente os dados de entrada para o modelo, e obtenha a previsão    
      loss = criterion(Y_pred.view(-1, 6), Y_train.view(-1))       # calcule a perda (o custo, o erro)
      optimizer.zero_grad()                   # inicialize os gradientes
      loss.backward()                         # backpropagation sobre a perda atual (cálculo dos novos gradientes) 
      optimizer.step()                        # atualização dos parâmetros da rede utilizando a regra do otimizador escolhido
      # FIM DO WORKFLOW DO TREINAMENTO    #
      #-----------------------------------#

      # ------ Bloco Opcional ------ #
      # Salvando métricas
      losses.append(loss)                     # salvando a perda atual
      acc = calc_accuracy(Y_pred, Y_train)     # calcula a taxa de acerto atual
      accs.append(acc)
      
      # Imprimindo resultados parciais
      if i % print_iters ==0: # a cada 10 iterações
        print(f'epoch: {i:2}  loss: {loss.item():10.8f}') 
      #-----------------------------------#

  #----------------------------------------------------------------------------# 
  print('\n# Finished training!')
  print(f'# --> epoch: {i}  \n# --> initial loss: {losses[0]:10.8f} ,  \n# --> accuracy: {acc:2.8f} , \n# --> final loss: {losses[-1]:10.8f}')
  
  # retornando resultados
  return model, losses, accs

# Redefinindo cálculo da taxa de acerto 
def calc_accuracy(y_pred, y_true):
  ''' Helper function para calcular a taxa de acerto deste exemplo.
  '''
  y_pred = torch.argmax(y_pred, dim=1)
  y_pred = y_pred.float()
  y_true = torch.squeeze(y_true) # tentar rexplicar dps
  y_pred = torch.squeeze(y_pred)
  num_hits  = torch.sum(y_pred==y_true).numpy()
  num_total =  float(y_true.numel())
  acc=  num_hits/num_total
  return acc

Now, we're convert everything to Tensors and instantiate our loss function

In [ ]:
x_train = np.vstack(X_train)
y_train = np.array(Y_train).reshape(-1,1)
x_train.shape, x_train[0].shape, y_train.shape, y_train[0].shape

In [ ]:
import tensorflow as tf

#data_train = (tf.constant(x_train, dtype=tf.float64), tf.constant(y_train, dtype=tf.float64))
data_train = (torch.float32(X_train), torch.float32(Y_train))

Model = mood_classifier(vocab_size=len(word2idx), dim_embed=100, n_units=100)
print(Model)

criterion = nn.NLLLoss() # cross entropy loss
optimizer = torch.optim.Adam(Model.parameters(), lr = 0.01) 

Model, losses, accs = train_loop(Model, data_train, max_epochs=330, print_iters=1) # note que o modelo é sobrescrito pela saída treinada

--------------------------------------------------------------------------------------------------

In [ ]:
X_train = torch.LongTensor(X_train)
Y_train = torch.LongTensor(Y_train)
X_test = np.vstack(X_test)
X_test = torch.LongTensor(X_test)
Y_test = np.array(Y_test).reshape(-1,1)

In [ ]:
Y_pred = torch.exp(Model.forward(X_test))         

In [ ]:
Y_pred = torch.argmax(Y_pred, dim=1)

In [ ]:
dataset["Feeling"].value_counts()

In [ ]:
dataset["emotion_code"].value_counts()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

sns.set(font_scale=1.5)

matriz_de_confusao = confusion_matrix(Y_pred, Y_test)

#criando list com as emoções
emotion_class = ['Angry','Disgust','Fear','Happy','Sad','Surprise']

df_matriz_de_confusao = pd.DataFrame(matriz_de_confusao, emotion_class, emotion_class)

# confusion matrix daora TEM Q MELHORAR
def plot_conf_mat(y_test, y_preds, norm='true'):
   fig, ax = plt.subplots(figsize=(8, 6))
   ax = sns.heatmap(df_matriz_de_confusao,
                   annot=True,
                    fmt="d",
                    cmap="YlOrRd")
   plt.xlabel("Resultado previsto")
   plt.ylabel("Resultado real")


plot_conf_mat(Y_test, Y_pred)

NICE GRAPHICS

In [ ]:
def plot_loss_and_accuracy(losses, accs):

  fig, ax_tuple = plt.subplots(1, 2, figsize=(16,6))
  fig.suptitle('Loss and accuracy')

  for i, (y_label, y_values) in enumerate(zip(['CE loss','Accuracy'],[losses, accs])):
    ax_tuple[i].plot(range(len(y_values)),  y_values, label='train')
    ax_tuple[i].set_xlabel('epochs')
    ax_tuple[i].set_ylabel(y_label)
    ax_tuple[i].legend()

In [ ]:
plot_loss_and_accuracy(losses, accs)

Instantiating another model

In [ ]:
#model = torch.load("/content/drive/My Drive/Análise de Sentimentos/Projeto para 08-09/Models/model.pth")
#model.eval()

In [ ]:
#data_train = (torch.LongTensor(x_train), torch.LongTensor(y_train))

#model = model(model, vocab_size=len(word2idx), dim_embed=100, n_units=100)

#criterion = nn.NLLLoss() # cross entropy loss
#optimizer = torch.optim.Adam(Model.parameters(), lr = 0.01) 

#model, losses, accs = train_loop(model, data_train, max_epochs=100, print_iters=1) # note que o modelo é sobrescrito pela saída treinada

https://keras.io/examples/nlp/pretrained_word_embeddings/